<h1 style="color:DarkTurquoise"> CrypTen Introduction / First Attempt </h1> 

This file contains code and notes for running the first implementation of CrypTen that I have tried. 


**NOTE**: once initial attempt is complete, please move this back into the crypten_case_studies folder within the driverdrowsiness application folder

In [1]:
from models.crypten_compactcnn import CryptenCompactCNN
import torch
import crypten
import crypten.mpc as mpc 
import crypten.communicator as comm 
import numpy as np

In [2]:
# define the parties involved
ALICE = 0
BOB = 1

crypten.init()
torch.set_num_threads(1)

In [3]:
crypten.common.serial.register_safe_class(CryptenCompactCNN)

In [15]:
# load and encrypt network architecture
dummy_model = CryptenCompactCNN()
path = "pretrained/sub9/model.pth"

# using CryptenCompactCNN simply replaces the way in which average pooling functions
plaintext_model = CryptenCompactCNN()
plaintext_model.load_state_dict(torch.load(path, map_location="cpu"))
print(plaintext_model)
dummy_input = torch.empty((1,1,1,384))

private_model = crypten.nn.from_pytorch(plaintext_model, dummy_input)
private_model.encrypt(src=ALICE)
print("Model successfully encrypted:", private_model.encrypted)

CryptenCompactCNN(
  (conv): Conv2d(1, 32, kernel_size=(1, 64), stride=(1, 1))
  (batch): Batchlayer()
  (fc): Linear(in_features=32, out_features=2, bias=True)
  (softmax): LogSoftmax(dim=1)
)
Model successfully encrypted: True


/home/jjl20011/snap/snapd-desktop-integration/83/Lab/Projects/Project1-V2X-Secure2PC/v2x-delphi-2pc/case_studies/driverdrowsiness/models/crypten_compactcnn.py:69: TracerWarning: Converting a tensor to a Python integer might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  b = torch.div(data - mean, variance).expand(int(data.size(0)),
/home/jjl20011/snap/snapd-desktop-integration/83/Lab/Projects/Project1-V2X-Secure2PC/v2x-delphi-2pc/case_studies/driverdrowsiness/models/crypten_compactcnn.py:70: TracerWarning: Converting a tensor to a Python integer might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  int(data.size(1)), int(data.size(2)), int(data.size(3)))
/home/jjl20011/snap/

In [4]:
# separate the labels and the data points themselves into separate ".pth" files to be compatible with CrypTen's system
from sklearn.metrics import accuracy_score

labels = torch.load("dev_work/test_features_labels/9-crypten_labels.pth")
labels.size()

torch.Size([314])

In [34]:
features = crypten.load_from_party("dev_work/test_features_labels/9-crypten_features.pth", src=ALICE)
features.size()

torch.Size([314, 1, 1, 384])

In [5]:
@mpc.run_multiprocess(world_size=2)
def sequential_eval():
    '''
    iteratively perform inference (as with Delphi)
    '''
    path = "pretrained/sub9/model.pth"
    # load pre-trained model to Alice
    plaintext_model = CryptenCompactCNN()
    plaintext_model.load_state_dict(torch.load(path, map_location="cpu"))
    print(plaintext_model)
    # fitting a single input for each run of the model
    dummy_input = torch.empty((1,1,1,384))

    private_model = crypten.nn.from_pytorch(plaintext_model, dummy_input)
    private_model.encrypt(src=ALICE)

    # load data to Bob
    features = crypten.load_from_party("dev_work/test_features_labels/9-crypten_features.pth", src=BOB)
    # is this step necessary? try it out once the multiprocess thing works properly
    pred_array = np.zeros(len(features))
    current = 0
    for i in range(len(features)): 
        current += 1
        if current > 15:
            print("Progress = {}".format(i/len(features)))
            current = 0
        # print(features[i].reshape((1,1,1,384)).size())
        output_enc = private_model(features[i])
        pred = output_enc.get_plain_text()
        pred = pred.argmax(axis=-1)
        # print(pred)
        if labels[i] == pred[0]: pred_array[i] = 1
        else: pred_array[i] = 0
    acc = np.sum(pred_array)/len(features)
    print("accuracy = {}".format(acc))

sequential_eval()

CryptenCompactCNN(
  (conv): Conv2d(1, 32, kernel_size=(1, 64), stride=(1, 1))
  (batch): Batchlayer()
  (fc): Linear(in_features=32, out_features=2, bias=True)
  (softmax): LogSoftmax(dim=1)
)
CryptenCompactCNN(
  (conv): Conv2d(1, 32, kernel_size=(1, 64), stride=(1, 1))
  (batch): Batchlayer()
  (fc): Linear(in_features=32, out_features=2, bias=True)
  (softmax): LogSoftmax(dim=1)
)


/home/jjl20011/snap/snapd-desktop-integration/83/Lab/Projects/Project1-V2X-Secure2PC/v2x-delphi-2pc/case_studies/driverdrowsiness/models/crypten_compactcnn.py:69: TracerWarning: Converting a tensor to a Python integer might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  b = torch.div(data - mean, variance).expand(int(data.size(0)),
/home/jjl20011/snap/snapd-desktop-integration/83/Lab/Projects/Project1-V2X-Secure2PC/v2x-delphi-2pc/case_studies/driverdrowsiness/models/crypten_compactcnn.py:70: TracerWarning: Converting a tensor to a Python integer might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  int(data.size(1)), int(data.size(2)), int(data.size(3)))
/home/jjl20011/snap/

Progress = 0.04777070063694268Progress = 0.04777070063694268

Progress = 0.09872611464968153Progress = 0.09872611464968153

Progress = 0.14968152866242038Progress = 0.14968152866242038

Progress = 0.20063694267515925
Progress = 0.20063694267515925
Progress = 0.2515923566878981Progress = 0.2515923566878981

Progress = 0.30254777070063693Progress = 0.30254777070063693

Progress = 0.3535031847133758Progress = 0.3535031847133758

Progress = 0.40445859872611467Progress = 0.40445859872611467

Progress = 0.4554140127388535Progress = 0.4554140127388535

Progress = 0.5063694267515924Progress = 0.5063694267515924

Progress = 0.5573248407643312Progress = 0.5573248407643312

Progress = 0.60828025477707Progress = 0.60828025477707

Progress = 0.6592356687898089Progress = 0.6592356687898089

Progress = 0.7101910828025477Progress = 0.7101910828025477

Progress = 0.7611464968152867Progress = 0.7611464968152867

Progress = 0.8121019108280255Progress = 0.8121019108280255

Progress = 0.8630573248407644Pro

[None, None]

In [7]:
(4*60 + 26.1)/314

0.8474522292993631

In [6]:
def compute_accuracy(output, labels):
    # need to convert this to cpu or numpy? experiment once multiprocess decorator begins to work properly
    pred = output.argmax(axis=-1)
    correct = pred.eq(labels)
    correct_count = correct.sum(0, keepdim=True).float()
    accuracy = correct_count.mul_(100.0 / output.size(0))
    return accuracy

@mpc.run_multiprocess(world_size=2)
def evaluate_model():
    path = "pretrained/sub9/model.pth"
    # load pre-trained model to Alice
    plaintext_model = CryptenCompactCNN()
    plaintext_model.load_state_dict(torch.load(path, map_location="cpu"))
    print(plaintext_model)
    dummy_input = torch.empty((314,1,1,384))

    private_model = crypten.nn.from_pytorch(plaintext_model, dummy_input)
    private_model.encrypt(src=ALICE)
    
    # load data to Bob
    features = crypten.load_from_party("dev_work/test_features_labels/9-crypten_features.pth", src=BOB)
    # is this step necessary? try it out once the multiprocess thing works properly
    # flattened_features = features.flatten(start_dim=1)

    # classify the encrypted data
    private_model.eval()
    output_enc = private_model(features)

    # compute the accuracy
    output = output_enc.get_plain_text()
    crypten.print(output.size())
    accuracy = compute_accuracy(output, labels)
    crypten.print("\tAccuracy: {0:.4f}".format(accuracy.item()))

evaluate_model()

CryptenCompactCNN(
  (conv): Conv2d(1, 32, kernel_size=(1, 64), stride=(1, 1))
  (batch): Batchlayer()
  (fc): Linear(in_features=32, out_features=2, bias=True)
  (softmax): LogSoftmax(dim=1)
)
CryptenCompactCNN(
  (conv): Conv2d(1, 32, kernel_size=(1, 64), stride=(1, 1))
  (batch): Batchlayer()
  (fc): Linear(in_features=32, out_features=2, bias=True)
  (softmax): LogSoftmax(dim=1)
)


/home/jjl20011/snap/snapd-desktop-integration/83/Lab/Projects/Project1-V2X-Secure2PC/v2x-delphi-2pc/case_studies/driverdrowsiness/models/crypten_compactcnn.py:69: TracerWarning: Converting a tensor to a Python integer might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  b = torch.div(data - mean, variance).expand(int(data.size(0)),
/home/jjl20011/snap/snapd-desktop-integration/83/Lab/Projects/Project1-V2X-Secure2PC/v2x-delphi-2pc/case_studies/driverdrowsiness/models/crypten_compactcnn.py:70: TracerWarning: Converting a tensor to a Python integer might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  int(data.size(1)), int(data.size(2)), int(data.size(3)))
/home/jjl20011/snap/

torch.Size([314, 2])

Process Process-4:


Process Process-3:
Traceback (most recent call last):
  File "/home/jjl20011/miniconda3/envs/crypten_env/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/home/jjl20011/miniconda3/envs/crypten_env/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/jjl20011/miniconda3/envs/crypten_env/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/jjl20011/miniconda3/envs/crypten_env/lib/python3.7/site-packages/crypten/mpc/context.py", line 30, in _launch
    return_value = func(*func_args, **func_kwargs)
  File "/home/jjl20011/miniconda3/envs/crypten_env/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipykernel_3293340/744888656.py", line 33, in evaluate_model
    accuracy = compute_accuracy(output, labels)
  File "/home/jjl20011/miniconda3/env

**Trouble Shooting**

One main issue with CrypTen arose while working with the tutorials `@mpc.run_multiprocess(world_size=x)` was `RuntimeError: CUDA Initialization Error`. This can be resolved (I think) by using the following series of commands: 

```bash
conda env config vars set CONDA_VISIBLE_DEVICES=""
conda deactivate
conda activate crypten_env
```

These commands simply set the conda environment variable, deactivate and then reactivate the conda environment - this is necessary to make sure that the environment variables are set properly

<h3 style="color:LightSalmon"> 